# 다양한 Prompt 종류

`prompt template`은 모델에 입력되는 텍스트 템플릿을 의미하며, 특정 작업을 수행하도록 모델을 안내하는 역할을 합니다.

LangChain은 프롬프트를 구성하고 작업하는 데 도움이 되는 여러 클래스와 기능을 제공합니다.

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) 

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

llm = ChatOpenAI(model="gpt-4o-mini")

### StringPromptTemplates

**문자열 프롬프트 템플릿(String PromptTemplates)** 은 단일 문자열을 포맷팅하는 데 사용되며, 일반적으로 간단한 입력에 활용됩니다.

In [4]:
from langchain_core.prompts import PromptTemplate

# 문자열 prompt template
prompt = """
저는 여행을 좋아해서 6개국을 다녀왔습니다.
조만간 몇 군데 더 방문할 계획입니다.

{query}
"""

prompt_template = PromptTemplate(
    input_variable=["query"],
    template=prompt
)

chain = prompt_template | llm
response = chain.invoke({"query": "위 내용에 대해 10단어로 트윗용 게시물을 작성할 수 있나요?"})
print(response.content)

여행 사랑, 6개국 방문! 더 많은 곳 계획 중! ✈️🌍


## PromptTemplate vs. ChatPromptTemplate

- `PomptTemplate.from_template()`는 템플릿에서 프롬프트를 생성합니다. 템플릿은 프롬프트의 기본 구조를 정의하고 프롬프트에 삽입될 변수를 포함할 수 있습니다.
- `ChatPromptTemplate.from_messages()`는 대화 메시지에서 프롬프트를 생성합니다. 대화 메시지는 사용자와 대화 시스템 간의 대화를 기록합니다.

### 1) PromptTemplate을 사용하여 문자열 프롬프트에 대한 템플릿을 생성합니다.  

python 의 `.format`, `f-string` 또는 langchain의 prompt template 을 사용할 수 있습니다.

1) PromptTemplate 클래스의 `from_template` 메서드를 사용하여 템플릿을 정의하고, format 메서드로 변수 값을 채워넣는 방식

In [6]:
# 'content'와 'adjective' 변수를 사용하여 프롬프트를 동적으로 구성
prompt_template = PromptTemplate.from_template(
    "{content}에 대해서 {adjective} 농담을 해줘 ."
)
# our_prompt = prompt_template.format(adjective="funny", content="chickens")
#our_prompt = prompt_template.format(adjective="재미있는", content="호랑이")
our_prompt = prompt_template.format(adjective="썰렁한", content="호랑이")
our_prompt

'호랑이에 대해서 썰렁한 농담을 해줘 .'

In [7]:
print(llm.invoke(our_prompt).content)

호랑이가 숲 속에서 맥주를 마시고 있었어요. 지나가는 사람이 물었어요. "왜 숲에서 맥주를 마시고 있어요?" 호랑이가 대답했어요. "스트레스가 많아서 '호랑이 기운'을 내야 해!"


2) `f-string` 을 사용하여 문자열 내에 직접 변수를 삽입

In [8]:
# 요약할 단어 수 설정
word_count = '3'

# 프롬프트 생성
# 'our_prompt' 변수의 내용과 'word_count' 변수를 사용하여 요약 요청을 구성
prompt = f"""{our_prompt}

이것을 {word_count} 단어 이내로 요약해 줘.
"""

print(prompt)
print("------------------------------------")
print(llm.invoke(prompt).content)

호랑이에 대해서 썰렁한 농담을 해줘 .

이것을 3 단어 이내로 요약해 줘.

------------------------------------
호랑이의 점수.


3) `Prompt Template` 클래스를 이용하여 입력 변수를 지정하고, 템플릿을 정의한 후 값을 채워넣는 방식

In [9]:
# 프롬프트 템플릿 정의
# 'our_text'와 'word_count' 변수를 사용하여 요약 요청을 구성합니다.
template = """
{our_text}

이것을 {word_count} 단어 이내로 요약해 줘.
"""

# PromptTemplate 객체 생성
# 'input_variables'에 'our_text'와 'word_count'를 지정하여 동적 입력을 받습니다.
prompt = PromptTemplate(
    input_variables=["our_text", "word_count"],
    template=template,
)

final_prompt = prompt.format(word_count='3', our_text=our_prompt)
final_prompt

'\n호랑이에 대해서 썰렁한 농담을 해줘 .\n\n이것을 3 단어 이내로 요약해 줘.\n'

In [10]:
llm.invoke(final_prompt).content

'호랑이 농담!'

### 2) ChatPromptTemplate을 사용하여 문자열 프롬프트에 대한 템플릿을 생성합니다.  
- 대화 메시지는 사용자와 대화 시스템 간의 대화를 기록. 따라서 프롬프트는 사용자의 질문과 대화 시스템의 답을 모두 포함

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

- 언어 모델에 직접 Message 목록을 전달

In [12]:
system_message = """당신은 영어 선생님입니다. 사용자는 영어 문장을 입력으로 제공합니다.
문장이 문법적으로 맞으면 맞음"을 출력하고, 문장이 문법적으로 틀리면 "틀림"을 출력하고
다음 줄에 틀린 문법이 수정된 문장을 출력 합니다."""

# human_message = "Get out house my you !"
# human_message = "You're beautiful."
human_message = "He have done this task."

messages = [
    SystemMessage(system_message),
    HumanMessage(human_message)
]

answer = llm.invoke(messages)
print(answer.content)

틀림  
He has done this task.


----
- ChatPromptTemplate을 사용해도 동일한 결과를 얻을 수 있습니다.

In [13]:
prompt_template = ChatPromptTemplate.from_messages(
    [('system', system_message), ('user', '{text}')]
)

prompt = prompt_template.invoke({'text': human_message})
answer = llm.invoke(prompt)
print(answer.content)

틀림  
He has done this task.


### Output Parser를 이용한 Chatting 출력 format 조정

In [14]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [15]:
# 시스템 메시지 프롬프트 템플릿을 정의
system_template = "당신은 입력 텍스트를 {output_language_pair}로 번역하는 유용한 번역사입니다. 각 언어의 출력을 쉼표로 구분합니다."
system_message_prompt = SystemMessage(system_template)
system_message_prompt

SystemMessage(content='당신은 입력 텍스트를 {output_language_pair}로 번역하는 유용한 번역사입니다. 각 언어의 출력을 쉼표로 구분합니다.', additional_kwargs={}, response_metadata={})

In [16]:
# 사용자 메시지 프롬프트 템플릿을 정의
human_template = "{text}"
human_message_prompt = HumanMessage(human_template)
human_message_prompt

HumanMessage(content='{text}', additional_kwargs={}, response_metadata={})

In [17]:
# 채팅 프롬프트 템플릿을 정의
chat_prompt = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", human_template)]
)
chat_prompt

ChatPromptTemplate(input_variables=['output_language_pair', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output_language_pair'], input_types={}, partial_variables={}, template='당신은 입력 텍스트를 {output_language_pair}로 번역하는 유용한 번역사입니다. 각 언어의 출력을 쉼표로 구분합니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [18]:
chain = chat_prompt | llm
print(chain.invoke({'text':'I am a boy', 'output_language_pair': '한국어, 일본어'}).content)
print(chain.invoke({'text':'I am a boy', 'output_language_pair': '중국어, 독일어'}).content)
print(chain.invoke({'text':'I am a boy', 'output_language_pair': '프랑스어, 베트남어'}).content)

나는 소년입니다, 私は男の子です
我是一个男孩, Ich bin ein Junge
Je suis un garçon, Tôi là một cậu bé.


### Chatting 출력의 감정 변경

In [19]:
# 시스템 메시지 템플릿 정의
# 'tone' 변수를 사용하여 전달받은 감정에 따라 텍스트를 재작성하도록 설정합니다.
system_template = "당신은 전달된 {tone} 감정에 따라 입력 텍스트를 다시 작성하는 유용한 도우미입니다."

# 시스템 메시지 객체 생성
system_message_prompt = SystemMessage(system_template)

# 사용자 메시지 템플릿 정의
# 'text' 변수를 사용하여 사용자가 입력한 텍스트를 전달합니다.
human_template = "{text}"

# 사용자 메시지 객체 생성
human_message_prompt = HumanMessage(human_template)

# ChatPromptTemplate을 사용하여 대화형 프롬프트 생성
# 시스템 메시지와 사용자 메시지를 순서대로 추가하여 대화 흐름을 구성합니다.
chat_prompt = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", human_template)]
)

# 최종 ChatPromptTemplate 객체 출력
chat_prompt

ChatPromptTemplate(input_variables=['text', 'tone'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tone'], input_types={}, partial_variables={}, template='당신은 전달된 {tone} 감정에 따라 입력 텍스트를 다시 작성하는 유용한 도우미입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [20]:
chain = chat_prompt | llm

print(chain.invoke({'text': '내가 사슴을 사냥했어요.', 'tone': '후회되는'}).content)
print()
print(chain.invoke({'text': '내가 사슴을 사냥했어요.', 'tone': '자랑스런'}).content)

사슴을 사냥한 경험이 안타깝게도 후회로 남았어요. 그 순간의 선택이 자연과 생명에 미친 영향을 깊이 생각하게 되네요.

저는 사슴을 사냥했어요! 정말 멋진 경험이었죠.


In [21]:
from langchain_core.prompts import PromptTemplate

# 요약 프롬프트 템플릿 정의
# 주어진 사람에 대한 정보를 바탕으로 요약과 흥미로운 사실을 생성하도록 설정합니다.
summary_template = """
    어떤 사람에 대한 정보 {information}를 제공하면 다음을 생성합니다.
     1. 짧은 요약
     2. 그 사람에 대한 두 가지 흥미로운 사실
"""

# PromptTemplate 객체 생성
# 'information' 변수를 입력으로 받아 프롬프트를 생성합니다.
summary_prompt_template = PromptTemplate(
    input_variables=["information"],
    template=summary_template
)

# 체인 생성
# 프롬프트 템플릿을 LLM(Language Model)과 연결하여 프롬프트를 처리합니다.
chain = summary_prompt_template | llm

# 입력 정보 설정
# 이순신 장군에 대한 상세한 정보를 입력합니다.
information = """
이순신(한국 한자: 李舜臣, 1545년 4월 28일 (음력 3월 8일) ~ 1598년 12월 16일 (음력 11월 19일))은 조선 중기의 무신이었다. 본관은 덕수(德水), 자는 여해(汝諧), 시호는 충무(忠武)였으며, 한성 출신이었다. 문반 가문 출신으로 1576년(선조 9년) 무과(武科)에 급제[2]하여 그 관직이 동구비보 권관, 훈련원 봉사, 발포진 수군만호, 조산보 만호, 전라남도수사를 거쳐 정헌대부 삼도수군통제사에 이르렀다.
함경도 동구비보권관(董仇非堡權管), 1581년 발포 수군만호(鉢浦水軍萬戶)가 되었다가 전라남수영의 오동나무를 베기를 거절하여 좌수사 성박의 미움을 받기도 했다. 이후 1584년 남병사의 군관과 건원보권관, 훈련원참군, 1586년 사복시주부를 거쳐 조산보만호 겸 녹도둔전사의(造山堡萬戶兼鹿島屯田事宜)로 부임했다. 조산만호 겸 녹둔도사의 재직 중 1587년(선조 20년) 9월의 여진족의 사전 기습공격으로 벌어진 녹둔도전투에서 이겼지만 피해가 커서, 북병사 이일의 탄핵을 받고 백의종군(白衣從軍)하는 위치에 서기도 했다. 그 뒤 두번째 여진족과의 교전에서 승전, 복직하였다. 그 뒤 전라관찰사 이광(李洸)에게 발탁되어 전라도 조방장, 선전관 등을 역임했다. 1589년 정읍현감 재직 중 유성룡의 추천으로 고사리첨사(高沙里僉使)가 되고, 절충장군(折衝將軍), 만포진첨사(滿浦鎭僉使), 진도군수 등을 거쳐 전라남도수사가 되어 임진왜란을 만나게 되었다.
임진왜란 때 조선의 삼도수군통제사가 되어 부하들을 통솔하는 지도력, 뛰어난 지략, 그리고 탁월한 전략과 능수능란한 전술로 일본 수군과의 해전에서 연전연승해 나라를 구한 성웅(聖雄)으로 추앙받고 있다. 노량 해전에서 전사한 뒤 선무공신 1등관에 추록되고 증 의정부우의정에 추증되고 덕풍군에 추봉되었다가, 광해군 때 다시 증 의정부좌의정에 추증되고 덕풍부원군에 추봉되었고, 정조 때에는 증 의정부영의정으로 가증(加贈)되었다.
고려 때 정5품 중랑장(中郎將)을 지낸 덕수 이씨의 시조 이돈수(李敦守)의 12대손이며, 조선 초 영중추부사(領中樞府事)를 지낸 이변(李邊)[3]의 후손이다. 외가는 초계 변씨(卞氏), 처가는 온양 방씨(方氏, 당시에는 상주 방씨)이다. 그의 묘는 충청남도 아산시에 있다.
"""

# 체인 실행
# 'information' 변수를 LLM에 전달하여 요약 및 흥미로운 사실을 생성합니다.
response = chain.invoke({"information": information})

# 결과 출력
print(response)

content='1. **짧은 요약**: 이순신(1545-1598)은 조선 중기의 무신으로, 임진왜란 동안 삼도수군통제사로서 일본 수군과 여러 차례의 해전에서 승리하며 나라를 구한 성웅으로 추앙받는다. 그는 노량 해전에서 전사한 후, 여러 차례의 추증과 봉작을 받았으며, 그의 묘는 충청남도 아산시에 위치하고 있다.\n\n2. **두 가지 흥미로운 사실**:\n   - 이순신은 문반 가문 출신으로 시작했지만, 1576년 무과에 급제하여 군인으로서의 경력을 쌓기 시작했다. 이는 그가 군사적 재능을 발휘하게 되는 중요한 전환점이었다.\n   - 그는 여진족과의 전투에서 두 번의 교전을 경험하며 승리한 후 복직되었는데, 이는 그의 전투 능력뿐만 아니라 정치적 역량도 강조하는 사례이다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 228, 'prompt_tokens': 877, 'total_tokens': 1105, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None} id='run-186a088f-b88f-4984-8ec8-48a15f397a2c-0' usage_metadata={'input_tokens': 877, 'output_tokens': 228, 'total_tokens': 1105, 'input_token_deta

In [22]:
print(response.content)

1. **짧은 요약**: 이순신(1545-1598)은 조선 중기의 무신으로, 임진왜란 동안 삼도수군통제사로서 일본 수군과 여러 차례의 해전에서 승리하며 나라를 구한 성웅으로 추앙받는다. 그는 노량 해전에서 전사한 후, 여러 차례의 추증과 봉작을 받았으며, 그의 묘는 충청남도 아산시에 위치하고 있다.

2. **두 가지 흥미로운 사실**:
   - 이순신은 문반 가문 출신으로 시작했지만, 1576년 무과에 급제하여 군인으로서의 경력을 쌓기 시작했다. 이는 그가 군사적 재능을 발휘하게 되는 중요한 전환점이었다.
   - 그는 여진족과의 전투에서 두 번의 교전을 경험하며 승리한 후 복직되었는데, 이는 그의 전투 능력뿐만 아니라 정치적 역량도 강조하는 사례이다.


## Few-shot prompt template

- 모델 프롬프트에 예제 입력과 예상 출력을 추가하는 기법  

- 모델 성능을 개선하는 가장 효과적인 방법 중 하나는 원하는 작업의 예를 모델에 제공하는 것입니다. 

---
몇 가지 샷 예제를 문자열로 포맷하는 포매터를 구성합니다. 이 포매터는 PromptTemplate 객체여야 합니다.

In [23]:
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("질문: {question}\n{answer}")

# example_prompt = PromptTemplate(
#     input_variables=["question", "answer"], 
#     template="질문: {question}\n{answer}")

example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='질문: {question}\n{answer}')

few-shot 예제 목록을 만들 것입니다. 각 예제는 위에서 정의한 포맷터 프롬프트에 대한 예제 입력을 나타내는 dictionary 이어야 합니다.  

주어진 예제는 COT (Chain of Thought) 방식으로 설계된 예제입니다. **COT (Chain of Thought)**는 복잡한 문제를 해결하기 위해 중간 단계의 추론 과정을 명시적으로 나열하는 방법입니다. 이 방식은 언어 모델이 단계별로 논리적 추론을 수행하도록 유도하여 더 정확한 결과를 도출할 수 있게 합니다.  

COT (Chain of Thought)의 핵심 요소
1. 후속 질문 (Follow-up Questions) - 질문에 대한 답변을 찾기 위해 필요한 추가 정보를 단계별로 요청합니다.
2. 중간 답변 (Intermediate Answers) - 각 단계에서 얻은 결과를 명확하게 기록합니다.
3. 최종 답변 (Final Answer) - 단계별 추론을 통해 얻은 결과를 종합하여 최종 답변을 도출합니다.

In [24]:
examples = [
   {
     "question": "무하마드 알리와 앨런 튜링 중 누가 더 오래 살았나요?",
     "answer":
"""
여기에 후속 질문이 필요합니까? 예.
후속 조치: 무하마드 알리가 사망했을 때 몇 살이었나요?
중간 답변: 무하마드 알리는 74세의 나이로 사망했습니다.
후속 조치: Alan Turing이 사망했을 때 몇 살이었습니까?
중간 답변: 앨런 튜링은 41세에 사망했습니다.
최종 답변은 다음과 같습니다. 무하마드 알리
"""
   },
   {
     "question": "craigslist의 창립자는 언제 태어났나요?",
     "answer":
"""
여기에 후속 질문이 필요합니까? 예.
후속 조치: craigslist의 창립자는 누구였나요?
중간 답변: Craigslist는 Craig Newmark가 설립했습니다.
후속 조치: Craig Newmark는 언제 태어났나요?
중간 답변: Craig Newmark는 1952년 12월 6일에 태어났습니다.
따라서 최종 답은 1952년 12월 6일입니다.
"""
   },
   {
     "question": "조지 워싱턴의 외할아버지는 누구였나요?",
     "answer":
"""
여기에 후속 질문이 필요합니까? 예.
후속 조치: 조지 워싱턴의 어머니는 누구였나요?
중간 답변: 조지 워싱턴의 어머니는 메리 볼 워싱턴(Mary Ball Washington)이었습니다.
후속 조치: 메리 볼 워싱턴(Mary Ball Washington)의 아버지는 누구였습니까?
중간 답변: Mary Ball Washington의 아버지는 Joseph Ball이었습니다.
최종 답변은 다음과 같습니다. Joseph Ball
"""
   },
   {
     "question": "Jaws와 Casino Royale의 디렉터는 모두 같은 국가 출신인가요?",
     "answer":
"""
여기에 후속 질문이 필요합니까? 예.
후속작: Jaws의 감독은 누구인가요?
중간 답변: 죠스의 감독은 스티븐 스필버그입니다.
후속 조치: 스티븐 스필버그는 어디 출신인가요?
중간 답변: 미국.
후속 조치: Casino Royale의 이사는 누구입니까?
중간 답변: Casino Royale의 이사는 Martin Campbell입니다.
후속 조치: Martin Campbell은 어디 출신인가요?
중간 답변: 뉴질랜드.
따라서 최종 대답은 다음과 같습니다. 아니요
"""
   }
]

In [25]:
# 포맷팅 프롬프트를 우리의 예시 중 하나로 테스트해 봅니다.
print(example_prompt.invoke(examples[0]).to_string())

질문: 무하마드 알리와 앨런 튜링 중 누가 더 오래 살았나요?

여기에 후속 질문이 필요합니까? 예.
후속 조치: 무하마드 알리가 사망했을 때 몇 살이었나요?
중간 답변: 무하마드 알리는 74세의 나이로 사망했습니다.
후속 조치: Alan Turing이 사망했을 때 몇 살이었습니까?
중간 답변: 앨런 튜링은 41세에 사망했습니다.
최종 답변은 다음과 같습니다. 무하마드 알리



In [26]:
from langchain_core.prompts import FewShotPromptTemplate

# FewShotPromptTemplate 객체 생성
prompt = FewShotPromptTemplate(
    examples=examples,              # 예제 데이터 리스트
    example_prompt=example_prompt,  # 각 예제를 포맷팅할 템플릿
    suffix="질문: {input}",      # 사용자 입력이 포함될 프롬프트의 마지막 부분
    input_variables=["input"]    # 프롬프트에서 사용될 변수 목록
)

# 체인 생성 - 프롬프트를 LLM(Language Model)과 연결하여 입력을 처리
chain = prompt | llm

# 체인 실행
response = chain.invoke("메리 볼 워싱턴의 아버지는 누구였나요??")

# 결과 출력
print(response)

content='여기에 후속 질문이 필요합니까? 예.  \n후속 조치: 메리 볼 워싱턴의 어머니는 누구였나요?  \n중간 답변: 메리 볼 워싱턴의 어머니는 Mary Johnson Ball입니다.  \n후속 조치: 메리 볼 워싱턴의 아버지는 언제 태어났나요?  \n중간 답변: Joseph Ball은 1649년에 태어났습니다.  \n최종 답변은 다음과 같습니다. Joseph Ball' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 562, 'total_tokens': 668, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None} id='run-d6d74924-cc76-48cf-81f9-ec066428c798-0' usage_metadata={'input_tokens': 562, 'output_tokens': 106, 'total_tokens': 668, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [27]:
print(chain.invoke("삼성그룹의 창립자는 언제 태어났나요?").content)

여기에 후속 질문이 필요합니까? 예.  
후속 조치: 삼성그룹의 창립자는 누구였나요?  
중간 답변: 삼성그룹은 이병철이 창립했습니다.  
후속 조치: 이병철은 언제 태어났나요?  
중간 답변: 이병철은 1910년 2월 12일에 태어났습니다.  
따라서 최종 답은 1910년 2월 12일입니다.


In [28]:
print(chain.invoke("현대 자동차와 테슬라 중에 어디가 더 큰 회사인가요?").content)

여기에 후속 질문이 필요합니까? 예.

후속 조치: 현대 자동차의 시가총액은 얼마인가요?  
중간 답변: 현대 자동차의 시가총액은 약 40조 원(약 350억 달러)입니다. (2023년 기준)  

후속 조치: 테슬라의 시가총액은 얼마인가요?  
중간 답변: 테슬라의 시가총액은 약 800억 달러입니다. (2023년 기준)  

최종 답변은 다음과 같습니다. 테슬라


In [29]:
print(chain.invoke("세종대왕과 단종임금 중에 누가 더 오래 살았나요?").content)

후속 질문이 필요합니까? 예.

후속 조치: 세종대왕은 몇 년에 태어났나요?  
중간 답변: 세종대왕은 1397년에 태어났습니다.

후속 조치: 세종대왕은 몇 년에 사망했나요?  
중간 답변: 세종대왕은 1450년에 사망했습니다.

후속 조치: 단종임금은 몇 년에 태어났나요?  
중간 답변: 단종임금은 1441년에 태어났습니다.

후속 조치: 단종임금은 몇 년에 사망했나요?  
중간 답변: 단종임금은 1457년에 사망했습니다.

최종 답변은 다음과 같습니다. 세종대왕
